In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

import requests

# Get data

In [2]:
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)

In [4]:
# get data files
download_url("https://cdn.freecodecamp.org/project-data/books/book-crossings.zip", './book-crossings.zip')

In [ ]:
!unzip book-crossings.zip

# Import data

In [12]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [13]:
book = "Where the Heart Is (Oprah's Book Club (Paperback))"

In [14]:
df_f = df_ratings.groupby('user').count().sort_values('isbn').reset_index()
users_200 = list(df_f[df_f['isbn'] >= 200]['user'])

In [162]:
df_b = df_ratings.groupby('isbn').count().sort_values('user')
books_g100 = df_b[df_b['user'] >= 100].index

In [167]:
df_r_f = df_ratings[
    (df_ratings['user'].isin(users_200)) &
    (df_ratings['isbn'].isin(df_books['isbn'].unique())) &
    (df_ratings['isbn'].isin(books_g100))]

In [168]:
df_book_feats = df_r_f.pivot(
index='isbn',
columns='user',
values='rating'
).fillna(0)

In [172]:
# create mapper from movie title to index
book_to_idx = {
b: i for i,b in
enumerate(list(df_books[df_books['isbn'].isin(books_g100)].set_index('isbn').loc[df_book_feats.index].title))
}

In [209]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  idx = book_to_idx[book]
  csm = csr_matrix(df_book_feats.values)

  model = NearestNeighbors(metric='cosine', algorithm='brute')
  model.fit(csm)

  resp = model.kneighbors(csm[book_to_idx[book]], n_neighbors = 5)
 
  recs = []
  for dist, idx in zip(resp[0][0][2:], resp[1][0][2:]):
    isbn = df_book_feats.iloc[idx].name
    title = df_books[df_books['isbn'] == isbn]['title'].values[0]
    if title == book:
      continue
    recs.append([title, dist]) 

  return [book, recs]

In [210]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['I Know This Much Is True', 0.7677075], ['The Surgeon', 0.7699411], ['The Weight of Water', 0.77085835]]]
You passed the challenge! 🎉🎉🎉🎉🎉
